In [89]:
import pandas as pd

In [98]:
jaws = pd.read_csv("data/global_shark_attacks.csv")
print(jaws.shape)

(25723, 24)


In [99]:
jaws.drop_duplicates(inplace=True)
jaws.sample()


,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
2536,1992.09.00,Sep-1992,1992.0,Unprovoked,USA,Florida,St Lucie County,Surf fishing,male,M,...,NaN,"Fort Pierce Tribune, 11/24/1992 & 11/30/1992",1992.09.00-surf-fisherman.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1992.09.00,1992.09.00,3767.0,NaN,NaN


In [100]:
jaws.drop(columns=['pdf', 'href formula', 'href', 'Unnamed: 22', 'Unnamed: 23', 'Case Number', 'Case Number.1','Case Number.2'], inplace=True)
jaws.head()

,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,original order
0,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",6303.0
1,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson�McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",6302.0
2,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",6301.0
3,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",6300.0
4,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,6299.0


In [101]:
jaws.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6312 entries, 0 to 25722
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    6302 non-null   object 
 1   Year                    6300 non-null   float64
 2   Type                    6298 non-null   object 
 3   Country                 6252 non-null   object 
 4   Area                    5847 non-null   object 
 5   Location                5762 non-null   object 
 6   Activity                5758 non-null   object 
 7   Name                    6092 non-null   object 
 8   Sex                     5737 non-null   object 
 9   Age                     3471 non-null   object 
 10  Injury                  6274 non-null   object 
 11  Fatal (Y/N)             5763 non-null   object 
 12  Time                    2948 non-null   object 
 13  Species                 3464 non-null   object 
 14  Investigator or Source  6285 non-null   obje

In [102]:
jaws.columns = [i.lower().replace(" ", "-") for i in jaws.columns]

In [103]:
jaws_renamed = jaws.rename(columns={"species-": "species", "sex-":"sex", "fatal-(y/n)":"fatal"})
jaws_renamed.head()

,date,year,type,country,area,location,activity,name,sex,age,injury,fatal,time,species,investigator-or-source,original-order
0,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",6303.0
1,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson�McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",6302.0
2,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",6301.0
3,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",6300.0
4,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,6299.0


In [104]:
jaws_renamed.species.unique()
s = "white" #risky?
jaws_renamed["species"] = [ s if s in str(i).lower() else i for i in jaws_renamed["species"] ]

t = "tiger" #risky?
jaws_renamed["species"] = [ t if t in str(i).lower() else i for i in jaws_renamed["species"] ]

len(jaws_renamed.species.unique())
#Oh my!!!

1069

In [105]:
jaws_renamed.type.unique()

array(['Boating', 'Unprovoked', 'Invalid', 'Provoked', 'Questionable',
       'Sea Disaster', nan, 'Boat', 'Boatomg'], dtype=object)

In [107]:
jaws_renamed.type.replace("Boatomg", "Boating", inplace=True)
jaws_renamed.type.replace("Boat", "Boating", inplace=True)
jaws_renamed.type.unique()

array(['Boating', 'Unprovoked', 'Invalid', 'Provoked', 'Questionable',
       'Sea Disaster', nan], dtype=object)

In [45]:
"""My feeble atempt
jaws_renamed.activity.replace("Crew swimming alongside their anchored ship", "Swimming", inplace=True)
jaws_renamed.activity.replace("4 men were bathing", "Swimming", inplace=True)
jaws_renamed.activity.replace("Swimming in pool formed by construction of a wharf", "Swimming", inplace=True)
jaws_renamed.activity.replace("Swimming around anchored ship", "Swimming", inplace=True)""" 
print(len(jaws_renamed.activity.unique()))

1529


In [108]:
#new atempt
n = "wreck"
jaws_renamed["activity"] = [ n if n in str(i) else i for i in jaws_renamed["activity"] ]
len(jaws_renamed.activity.unique())

1512

In [109]:
s = "swim"
jaws_renamed["activity"] = [ s if s in str(i).lower() else i for i in jaws_renamed["activity"] ]
b = "bathing"
jaws_renamed["activity"] = [ b if b in str(i).lower() else i for i in jaws_renamed["activity"] ]
o = "surf"
jaws_renamed["activity"] = [ o if o in str(i).lower() else i for i in jaws_renamed["activity"] ]
u = "fishing"
jaws_renamed["activity"] = [ u if u in str(i).lower() else i for i in jaws_renamed["activity"] ]
len(jaws_renamed.activity.unique())

931

In [110]:
""" Cleaning the column sex"""
jaws_renamed.sex.replace("M ", "M", inplace=True)
jaws_renamed.sex.unique()

array(['F', 'M', nan, 'lli', 'N', '.'], dtype=object)

In [113]:
""" Cleaning the column fatal"""
jaws_renamed.fatal.replace("y", "Y", inplace=True)
jaws_renamed.fatal.replace({" N" : "N", "N ":"N"}, inplace=True)

jaws_renamed.loc[jaws_renamed['sex'] == "N"]
#jaws_renamed.fatal.unique()

,date,year,type,country,area,location,activity,name,sex,age,injury,fatal,time,species,investigator-or-source,original-order
4938,11-Jul-1934,1934.0,Boating,AUSTRALIA,New South Wales,Cronulla,fishing,"18' boat, occupants William & Leslie Newton",N,NaN,No injury to occupants Sharks continually foll...,N,NaN,"Blue pointer, 11'","G.P. Whitley, ref: Daily Telegraph, 7/11/1934 ...",1365.0
6131,Reported 18-Dec-1801,1801.0,Provoked,NaN,NaN,NaN,Standing on landed shark's tail,Stephen Pettigew,N,NaN,"FATAL, PROVOKED INCIDENT",Y,NaN,12' shark,"The Evening Post, 12/18/1801",172.0


In [84]:
len(jaws_renamed.injury.unique())


3738